Flux with PCA

Libraries

In [ ]:
using MLDatasets           # mnist
using Images
using PreprocessingImages; pim = PreprocessingImages
using PreprocessingArrays; pa  = PreprocessingArrays

using MLJ                  # make_blobs, rmse, confmat, f1score, coerce
using MLDataUtils          # label, nlabel, labelfreq, stratifiedobs
using Flux                 # the julia ml library

using Random
using DataFrames

In [ ]:
include( expanduser("~/projects/pesquisa/libs/libml.jl") )

MNIST

In [ ]:
# load mnist
datasetX,    datasetY    = MNIST(:train)[:]
validationX, validationY = MNIST(:test)[:]

display( size(datasetX) )

img  = datasetX[:, :, 1:5]
img2 = permutedims(img, (2, 1, 3))

display(datasetY[1:5]')
mosaicview( Gray.(img2)  ; nrow=1)

In [ ]:
# split trainset, testset from dataset
Random.seed!(1)
(trainX, trainY), (testX, testY) = stratifiedobs((datasetX, datasetY), p = 0.7)
size(trainX), size(testX), size(validationX)

Preprocessing

In [ ]:
# MLJ works with dataframes
function preprocess1(X, y)
    Xs = permutedims(X, (2, 1, 3))
    Xs = pim.batchImage2DF(Xs)
    ys = Int32.(y)

    return (Xs, ys)
end

h, v, N = size(trainX); d = h * v
X_c, y_c = preprocess1(trainX, trainY)
d, N, size(X_c), size(y_c)

In [ ]:
# reduce predictors
PCA = @load PCA pkg=MultivariateStats verbosity=0
reducer = PCA(pratio = 0.95)

# standardize predictors
std = Standardizer()

# execute
pipe = @pipeline reducer std
mach = MLJ.machine(pipe, X_c) |> fit!
X_c = MLJ.transform(mach, X_c) .|> Float32   # transform(unsupervised) vs predict(supervised)
typeof(X_c)

In [ ]:
# Flux works with matices and vectors
function preprocess2(X, y)
    N, d = size(X)
    Xs = X |> Matrix 
    Xs = Float32.(Xs) |> Flux.flatten
    Xs = [Xs[i,:] for i in 1:N]
    Xs = Flux.batch(Xs)
    
    ys = Flux.onehotbatch( Float32.(y), 0:9 )
    
    return (Xs, ys)
end

X_c, y_c = preprocess2(X_c, y_c)
typeof(X_c)

Model

Stochastic Gradient Descent (SGD) means batchsize=1.

The error on every and each data line is computed, then weights are recalculated towards the negative of the gradient.

*** check *** For this reason, calculation is done at CPU (sequencial computation over one data point at a time).

In [ ]:
# model configuration
nInputs  = size(X_c)[1]
nOutputs = 10

model              = Chain( Flux.Dense(nInputs, nOutputs, tanh),   # tanh is chosen as nonlinearity (Prof Mostafa lecture)
                            softmax )                              # softmax scales the output to sum to one
lossFunction(X, y) = Flux.mse( model(X), y )
modelParameters    = Flux.params(model)
dataset            = Flux.DataLoader((X_c, y_c), batchsize=1)  # batchsize=1 => SGD, batch size > 1 => batch gradient descent
modelOptimizer     = Flux.Descent()
callBack           = Flux.throttle(() -> println("."), 10); # print every 10s

##### Training

In [ ]:
numberOfEpochs = 100

Training method without plotting epochs

In [ ]:
# # preferred for multiple epochs
# epochs = 1:numberOfEpochs
# for epoch in epochs
#     Flux.train!(lossFunction, modelParameters, data, modelOptimizer; cb=callBack)
# end

Training method with plotting, with the aid of homemade function trainModel!

In [ ]:
# https://fluxml.ai/Flux.jl/stable/training/training/

epochs     = 1 : numberOfEpochs
lossVector = Vector{Float64}()
minLoss    = 1e-5   # hint: start at 1e-2, then go lower for more epochs (5e-4)
nearZero   = minLoss / 25

for epoch in epochs
    # train epoch
    loss = trainModel!( lossFunction, modelParameters, dataset, modelOptimizer )   # libml
    push!(lossVector, loss)
    print(".")   # "epoch completed" indicator

    # exit criteria
    if stopTrainingCriteria(lossVector, minLoss, nearZero)   break   end
end

In [ ]:
# plot training
plotLoss(lossVector, (800,500))

##### Testing

In [ ]:
function predictOutcome(X)
    ŷ = Flux.onecold( model(X), [0:9;] )
end

In [ ]:
# preprocessing
X_ts, y_ts = preprocess1(testX, testY)
X_ts       = MLJ.transform(mach, X_ts)
X_ts, y_ts = preprocess2(X_ts, y_ts)

# predict
ŷ = predictOutcome(X_ts)

# result
printMetrics( ŷ, coerce(testY, OrderedFactor) )